<a href="https://colab.research.google.com/github/aniketSanyal/DifferentialPrivacy/blob/main/LORA_DP_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install loralib transformers datasets opacus

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("glue", "sst2")

In [ ]:
class Config:
  num_train_epochs = 4
  learning_rate = 5e-4
  n_prompt_tokens = 10
  random_range  = 0.5
  batch_size = 64
  max_grad_norm = 0.1
args = Config()

In [ ]:
from transformers import AutoTokenizer

checkpoint = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:


tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["sentence"], max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    input_ids = pad_sequence([item['input_ids'] for item in batch], batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence([item['attention_mask'] for item in batch], batch_first=True, padding_value=0)
    labels = torch.stack([item['labels'] for item in batch])

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [ ]:
train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = model.to(device)
model = model.train()

In [ ]:
import loralib as lora

In [ ]:
import torch.nn as nn
lora_r = 4

def make_lora_layer(layer, lora_r=4):
    new_layer = lora.Linear(
        in_features=layer.in_features,
        out_features=layer.out_features,
        bias=layer.bias is None,
        r=lora_r,
        merge_weights=False
    )

    new_layer.weight = nn.Parameter(layer.weight.detach())

    if layer.bias is not None:
        new_layer.bias = nn.Parameter(layer.bias.detach())

    return new_layer


def make_lora_replace(model, depth=1, path="", verbose=True):
    if depth > 10:
        return
    depth += 1

    if isinstance(model, nn.Linear) and "attention" in path:
        if verbose:
            print(f"Find linear {path}:{key} :", type(module))

        return make_lora_layer(model)

    for key in dir(model):
        module = getattr(model, key)
        module_type = type(module)

        if not isinstance(module, nn.Module) or module is model:
            continue

        if isinstance(module, nn.Linear) and "attention" in path:
            layer = make_lora_layer(module)
            setattr(model, key, layer)
            if verbose:
                print(f"Find linear {path}:{key} :", type(module))

        elif isinstance(module, nn.ModuleList):
            for i, elem in enumerate(module):
                layer = make_lora_replace(elem, depth, path+":"+key+f"[{i}]", verbose=verbose)
                if layer is not None:
                    module[i] = layer

        elif isinstance(module, nn.ModuleDict):
            for module_key in list(module.keys()):
                layer = make_lora_replace(elem, depth, path+":"+key+":"+module_key, verbose=verbose)
                if layer is not None:
                    module[module_key] = layer

        else:
            layer = make_lora_replace(module, depth, path+":"+key, verbose=verbose)
            if layer is not None:
                setattr(model, key, layer)

In [ ]:
make_lora_replace(model, verbose=True)

In [ ]:
lora_r = 4

total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters before LoRA: {total_trainable_params}")


lora.mark_only_lora_as_trainable(model)

total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters after LoRA: {total_trainable_params}")

In [ ]:
optimiser = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
model.train()

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,

)

In [ ]:
import numpy as np

def accuracy(preds, labels):
    return (preds == labels).mean()

# define evaluation cycle
def evaluate(model):
    model.eval()

    loss_arr = []
    accuracy_arr = []

    for batch in test_dataloader:
        #batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss, logits = outputs[:2]

        preds = np.argmax(logits.detach().cpu().numpy(), axis=1)
        labels =batch['labels'].detach().cpu().numpy()

        loss_arr.append(loss.item())
        accuracy_arr.append(accuracy(preds, labels))

    model.train()
    return np.mean(loss_arr), np.mean(accuracy_arr)

In [ ]:
from opacus.utils.batch_memory_manager import BatchMemoryManager

In [ ]:
LOGGING_INTERVAL = 100

In [ ]:
def train_LORA_DP(model, optimiser,train_loader, DELTA):
  for epoch in range(1, args.num_train_epochs+1):
    losses = []

    with BatchMemoryManager(
        data_loader=train_dataloader,
        max_physical_batch_size=16,
        optimizer=optimiser
    ) as memory_safe_data_loader:
        for step, batch in enumerate(memory_safe_data_loader):
            optimiser.zero_grad()
            #batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch) # output = loss, logits, hidden_states, attentions
            #print(outputs)
            loss = outputs[0]

            loss.backward()
            losses.append(loss.item())
            optimiser.step()

            if step > 0 and step % LOGGING_INTERVAL == 0:
                train_loss = np.mean(losses)
                eps = privacy_engine.get_epsilon(DELTA)
                eval_loss, eval_accuracy = evaluate(model)

                print(
                  f"Epoch: {epoch} | "
                  f"Step: {step} | "
                  f"Train loss: {train_loss:.3f} | "
                  f"Eval loss: {eval_loss:.3f} | "
                  f"Eval accuracy: {eval_accuracy:.3f} | "
                  f"ɛ: {eps:.2f}"
                )


In [ ]:
train_LORA_DP(model, optimiser, train_dataloader, DELTA)

For QNLI

In [ ]:
raw_datasets = load_dataset("glue", "qnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question"],example["sentence"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'sentence', 'question'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


In [ ]:
make_lora_replace(model, verbose=True)
lora_r = 4
lora.mark_only_lora_as_trainable(model)

In [ ]:
optimiser = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,

)

In [ ]:
train_LORA_DP(model, optimiser, train_dataloader, DELTA)

QQP

In [ ]:
raw_datasets = load_dataset("glue", "qqp")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation'] = raw_datasets['validation'].select([i for i in range(5000)])

raw_datasets['test'] = raw_datasets['test'].select([i for i in range(5000)])


raw_datasets['train'][:5]


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["question1"],example["question2"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'question1', 'question2'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

make_lora_replace(model, verbose=True)
lora_r = 4
lora.mark_only_lora_as_trainable(model)

optimiser = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
DELTA = 1/len(train_dataloader)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,

)

In [ ]:
train_LORA_DP(model, optimiser, train_dataloader, DELTA)

MNLI

In [ ]:
raw_datasets = load_dataset("glue", "mnli")


raw_datasets['train'] = raw_datasets['train'].select([i for i in range(50000)])

raw_datasets['validation_matched'] = raw_datasets['validation_matched'].select([i for i in range(5000)])



tokenizer = AutoTokenizer.from_pretrained(checkpoint, num_labels =3)


from torch.utils.data import DataLoader
tokenized_dataset = raw_datasets.map(
    lambda example: tokenizer(example["premise"],example["hypothesis"] ,max_length=64, padding='max_length', truncation=True),
    batched=True
)


tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenized_dataset = tokenized_dataset.remove_columns(['idx', 'premise', 'hypothesis'])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")


train_dataloader = DataLoader(tokenized_dataset["train"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)
test_dataloader = DataLoader(tokenized_dataset["validation_matched"], shuffle=False, batch_size=args.batch_size, collate_fn=custom_collate)

In [ ]:
model =AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

In [ ]:
make_lora_replace(model, verbose=True)
lora_r = 4
lora.mark_only_lora_as_trainable(model)

optimiser = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
privacy_engine = PrivacyEngine()
model.train()
model, optimiser, train_dataloader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimiser,
    data_loader=train_dataloader,
    target_delta=DELTA,
    target_epsilon=8,
    epochs=args.num_train_epochs,
    max_grad_norm=args.max_grad_norm,

)

In [ ]:
train_LORA_DP(model, optimiser, train_dataloader, DELTA)